In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
import os
from datetime import datetime, timedelta
import gmail_api
import base64
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import io
import logging

In [2]:
EQUITY_BASE = 'S3121200 - Equity Base'
WPAC_STATEMENT = 'Wpac Statement data'

trading_account = pd.read_excel('Wentworth Stock & Trading Futures Account.xls', sheet_name=EQUITY_BASE)

In [3]:
trading_account.tail()

,Business Date (Start),Starting Cash,Daily Charges,Profit and Loss,Daily Transfers,Daily Received Cash,Daily Cash Paid,Cash,Open Trade Equity,Total Equity,...,YTD Profit and Loss,Gross Trading P&L,Net Trading P&L,Open Trade Equity,Daily Futures P&L,Drawdown (Total FUM),Margin Utilsation,FUM,Wpac BizOne,Wpac Cash Reserve
302,2025-08-27,842333.63,-75.08,-5220.06,0.0,0,0,837038.49,0.00,837038.49,...,-6828.98,0.00,-75.08,3512.22,3437.14,0.000000,0.000000,888971.79,25159,26774.30
303,2025-08-28,837029.81,-150.22,0.00,0.0,0,0,836879.59,-715.34,836164.25,...,-6831.47,-715.34,-865.56,-715.34,-1580.90,-0.000827,0.060191,864592.89,939,26774.30
304,2025-08-29,836878.53,-25.02,0.00,0.0,0,0,836853.51,14565.15,851418.66,...,-6826.86,14565.15,14540.13,15280.49,29820.62,0.016846,0.070176,864581.56,939,26789.05
305,2025-09-01,836850.52,-150.12,18586.48,0.0,0,0,855286.88,10168.95,865455.83,...,11759.53,10168.95,10018.83,-4396.20,5622.63,0.011516,0.029448,883014.93,939,26789.05
306,2025-09-02,855286.88,0.00,0.00,0.0,0,0,855286.88,10168.95,865455.83,...,11759.53,10168.95,10168.95,0.00,10168.95,0.008598,0.000000,1182685.93,300610,26789.05


In [4]:
trading_account.columns

Index(['Business Date (Start)', 'Starting Cash', 'Daily Charges',
       'Profit and Loss', 'Daily Transfers', 'Daily Received Cash',
       'Daily Cash Paid', 'Cash', 'Open Trade Equity', 'Total Equity',
       'Net Liquidation Value', 'Initial Margin', 'Maintenance Margin',
       'Excess Deficit', 'MTD Profit and Loss', 'YTD Profit and Loss',
       'Gross Trading P&L', 'Net Trading P&L', 'Open Trade Equity ',
       'Daily Futures P&L', 'Drawdown (Total FUM)', 'Margin Utilsation ',
       'FUM', 'Wpac BizOne', 'Wpac Cash Reserve'],
      dtype='object')

In [5]:
# Database Configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'user': 'zeerakwyne',
    'password': '',
    'database': 'wentworth'
}

# Gmail Configuration (you'll need to set these up)
GMAIL_CREDENTIALS = {
    'credentials_file': 'credentials.json',  # Download from Google Cloud Console
    'token_file': 'token.json'  # Will be created after first auth
}

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [6]:
def create_database():
    """Create the wentworth database if it doesn't exist"""
    try:
        # Connect to default postgres database to create wentworth
        conn = psycopg2.connect(
            host=DB_CONFIG['host'],
            port=DB_CONFIG['port'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password'],
            database='postgres'  # Connect to default postgres db
        )
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Check if database exists
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = 'wentworth'")
        exists = cursor.fetchone()
        
        if not exists:
            cursor.execute("CREATE DATABASE wentworth")
            logger.info("Database 'wentworth' created successfully")
        else:
            logger.info("Database 'wentworth' already exists")
            
        cursor.close()
        conn.close()
        return True
        
    except Exception as e:
        logger.error(f"Error creating database: {e}")
        return False

# Test database creation
create_database()


2025-10-19 23:43:19,318 - INFO - Database 'wentworth' already exists


True

In [7]:
def generate_excel_from_database_updated(output_file=None):
    """Generate Excel file from database data - Updated to match original format"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        
        # Query all data from database
        query = """
        SELECT 
            business_date, starting_cash, daily_charges, profit_and_loss, 
            daily_transfers, daily_received_cash, daily_cash_paid, cash, 
            open_trade_equity, total_equity, net_liquidation_value, 
            initial_margin, maintenance_margin, excess_deficit, 
            mtd_profit_and_loss, ytd_profit_and_loss
        FROM equity_data 
        ORDER BY business_date DESC;
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        # Rename columns to match original Excel format exactly
        column_mapping = {
            'business_date': 'Business Date (Start)',
            'starting_cash': 'Starting Cash',
            'daily_charges': 'Daily Charges',
            'profit_and_loss': 'Profit and Loss',
            'daily_transfers': 'Daily Transfers',
            'daily_received_cash': 'Daily Received Cash',
            'daily_cash_paid': 'Daily Cash Paid',
            'cash': 'Cash',
            'open_trade_equity': 'Open Trade Equity',
            'total_equity': 'Total Equity',
            'net_liquidation_value': 'Net Liquidation Value',
            'initial_margin': 'Initial Margin',
            'maintenance_margin': 'Maintenance Margin',
            'excess_deficit': 'Excess Deficit',
            'mtd_profit_and_loss': 'MTD Profit and Loss',
            'ytd_profit_and_loss': 'YTD Profit and Loss'
        }
        
        df = df.rename(columns=column_mapping)
        
        # Add the additional columns that are in the original Excel but not in our database
        # These will be empty/NaN for now, but we can populate them later if needed
        additional_columns = {
            'Gross Trading P&L': df['Profit and Loss'],  # Same as Profit and Loss
            'Net Trading P&L': df['Profit and Loss'],    # Same as Profit and Loss  
            'Open Trade Equity ': df['Open Trade Equity'],  # Note the trailing space
            'Daily Futures P&L': df['Profit and Loss']   # Same as Profit and Loss
        }
        
        # Add the additional columns
        for col_name, col_data in additional_columns.items():
            df[col_name] = col_data
        
        # Reorder columns to match original Excel format
        column_order = [
            'Business Date (Start)', 'Starting Cash', 'Daily Charges', 'Profit and Loss',
            'Daily Transfers', 'Daily Received Cash', 'Daily Cash Paid', 'Cash',
            'Open Trade Equity', 'Total Equity', 'Net Liquidation Value', 'Initial Margin',
            'Maintenance Margin', 'Excess Deficit', 'MTD Profit and Loss', 'YTD Profit and Loss',
            'Gross Trading P&L', 'Net Trading P&L', 'Open Trade Equity ', 'Daily Futures P&L'
        ]
        
        df = df[column_order]
        
        # Generate output filename if not provided
        if output_file is None:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            output_file = f'../data/Wentworth_Stock_Trading_Futures_Account_{timestamp}.xlsx'
        
        # Create Excel file with multiple sheets
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Main equity data sheet
            df.to_excel(writer, sheet_name=EQUITY_BASE, index=False)
            
            # Summary sheet
            summary_data = {
                'Total Records': [len(df)],
                'Date Range': [f"{df['Business Date (Start)'].min()} to {df['Business Date (Start)'].max()}"],
                'Last Updated': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
                'Account ID': ['S3121200'],
                'Legal Account Name': ['WENTWORTH STOCK AND TRADING PTY LTD']
            }
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        logger.info(f"Excel file generated successfully: {output_file}")
        return output_file
        
    except Exception as e:
        logger.error(f"Error generating Excel file: {e}")
        return None

# Test the updated Excel generation
# generate_excel_from_database_updated()


In [8]:
def create_equity_table():
    """Create the equity_data table with proper schema"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        # Create table with all columns from CSV
        create_table_query = """
        CREATE TABLE IF NOT EXISTS equity_data (
            id SERIAL PRIMARY KEY,
            business_date DATE NOT NULL,
            managing_location VARCHAR(50),
            account_id VARCHAR(50),
            family_group_code_1 VARCHAR(50),
            legal_account_name VARCHAR(255),
            currency VARCHAR(10),
            starting_cash DECIMAL(15,2),
            daily_charges DECIMAL(15,2),
            daily_tax DECIMAL(15,2),
            daily_option_premiums DECIMAL(15,2),
            profit_and_loss DECIMAL(15,2),
            daily_transfers DECIMAL(15,2),
            daily_received_cash DECIMAL(15,2),
            daily_cash_paid DECIMAL(15,2),
            cash DECIMAL(15,2),
            open_trade_equity DECIMAL(15,2),
            total_equity DECIMAL(15,2),
            net_option_value DECIMAL(15,2),
            net_liquidation_value DECIMAL(15,2),
            initial_margin DECIMAL(15,2),
            maintenance_margin DECIMAL(15,2),
            excess_deficit DECIMAL(15,2),
            mtd_profit_and_loss DECIMAL(15,2),
            ytd_profit_and_loss DECIMAL(15,2),
            forward_cash_entries DECIMAL(15,2),
            forward_futures_pl DECIMAL(15,2),
            forward_charges DECIMAL(15,2),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(business_date, account_id)
        );
        """
        
        cursor.execute(create_table_query)
        
        # Create index on business_date for efficient querying
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_equity_business_date ON equity_data(business_date);")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_equity_account_id ON equity_data(account_id);")
        
        conn.commit()
        cursor.close()
        conn.close()
        
        logger.info("Equity data table created successfully")
        return True
        
    except Exception as e:
        logger.error(f"Error creating equity table: {e}")
        return False

# Create the table
create_equity_table()


2025-10-19 23:43:20,906 - INFO - Equity data table created successfully


True

In [9]:
# # Let's examine the actual Excel file structure
# excel_file = '../data/Wentworth Stock & Trading Futures Account.xls'
# df_excel = pd.read_excel(excel_file, sheet_name=EQUITY_BASE)

# print("Excel file shape:", df_excel.shape)
# print("\nColumn names:")
# print(df_excel.columns.tolist())
# print("\nFirst few rows:")
# print(df_excel.head())
# print("\nData types:")
# print(df_excel.dtypes)


def migrate_excel_to_database_fixed():
    """Migrate existing Excel data to PostgreSQL database - FIXED VERSION"""
    try:
        # Read the existing Excel file
        excel_file = 'Wentworth Stock & Trading Futures Account.xls'
        df = pd.read_excel(excel_file, sheet_name=EQUITY_BASE)
        
        logger.info(f"Loaded {len(df)} rows from Excel file")
        logger.info(f"Columns: {df.columns.tolist()}")
        
        # Clean the data - remove rows with null business dates
        df_clean = df.dropna(subset=['Business Date (Start)'])
        logger.info(f"After cleaning: {len(df_clean)} rows")
        
        # Connect to database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        # Prepare data for insertion - mapping Excel columns to database columns
        insert_query = """
        INSERT INTO equity_data (
            business_date, managing_location, account_id, family_group_code_1, 
            legal_account_name, currency, starting_cash, daily_charges, daily_tax,
            daily_option_premiums, profit_and_loss, daily_transfers, daily_received_cash,
            daily_cash_paid, cash, open_trade_equity, total_equity, net_option_value,
            net_liquidation_value, initial_margin, maintenance_margin, excess_deficit,
            mtd_profit_and_loss, ytd_profit_and_loss, forward_cash_entries,
            forward_futures_pl, forward_charges
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        ) ON CONFLICT (business_date, account_id) DO NOTHING;
        """
        
        # Convert DataFrame to list of tuples with proper mapping
        data_tuples = []
        for _, row in df_clean.iterrows():
            # Map Excel columns to database columns
            data_tuple = (
                row.get('Business Date (Start)'),  # business_date
                'SINSFT',  # managing_location (default value)
                'S3121200',  # account_id (default value)
                None,  # family_group_code_1
                'WENTWORTH STOCK AND TRADING PTY LTD',  # legal_account_name (default value)
                'AUD',  # currency (default value)
                row.get('Starting Cash'),  # starting_cash
                row.get('Daily Charges'),  # daily_charges
                None,  # daily_tax
                None,  # daily_option_premiums
                row.get('Profit and Loss'),  # profit_and_loss
                row.get('Daily Transfers'),  # daily_transfers
                row.get('Daily Received Cash'),  # daily_received_cash
                row.get('Daily Cash Paid'),  # daily_cash_paid
                row.get('Cash'),  # cash
                row.get('Open Trade Equity'),  # open_trade_equity
                row.get('Total Equity'),  # total_equity
                None,  # net_option_value
                row.get('Net Liquidation Value'),  # net_liquidation_value
                row.get('Initial Margin'),  # initial_margin
                row.get('Maintenance Margin'),  # maintenance_margin
                row.get('Excess Deficit'),  # excess_deficit
                row.get('MTD Profit and Loss'),  # mtd_profit_and_loss
                row.get('YTD Profit and Loss'),  # ytd_profit_and_loss
                None,  # forward_cash_entries
                None,  # forward_futures_pl
                None   # forward_charges
            )
            data_tuples.append(data_tuple)
        
        # Insert data
        cursor.executemany(insert_query, data_tuples)
        conn.commit()
        
        # Get count of inserted records
        cursor.execute("SELECT COUNT(*) FROM equity_data")
        count = cursor.fetchone()[0]
        
        cursor.close()
        conn.close()
        
        logger.info(f"Successfully migrated {count} records to database")
        return True
        
    except Exception as e:
        logger.error(f"Error migrating Excel data: {e}")
        return False

# Migrate the data with the fixed function
migrate_excel_to_database_fixed()


2025-10-19 23:43:21,898 - INFO - Loaded 307 rows from Excel file
2025-10-19 23:43:21,899 - INFO - Columns: ['Business Date (Start)', 'Starting Cash', 'Daily Charges', 'Profit and Loss', 'Daily Transfers', 'Daily Received Cash', 'Daily Cash Paid', 'Cash', 'Open Trade Equity', 'Total Equity', 'Net Liquidation Value', 'Initial Margin', 'Maintenance Margin', 'Excess Deficit', 'MTD Profit and Loss', 'YTD Profit and Loss', 'Gross Trading P&L', 'Net Trading P&L', 'Open Trade Equity ', 'Daily Futures P&L', 'Drawdown (Total FUM)', 'Margin Utilsation ', 'FUM', 'Wpac BizOne', 'Wpac Cash Reserve']
2025-10-19 23:43:21,900 - INFO - After cleaning: 307 rows
2025-10-19 23:43:21,973 - INFO - Successfully migrated 331 records to database


True

In [10]:
def setup_gmail_api():
    """Initialize Gmail API connection"""
    try:
        from gmail_api import GmailAPI
        gmail = GmailAPI(
            credentials_file='../credentials.json',
            token_file='../token.json'
        )
        logger.info("Gmail API setup successful")
        return gmail
    except Exception as e:
        logger.error(f"Error setting up Gmail API: {e}")
        return None

# Initialize Gmail API (you'll need to set up credentials first)
gmail = setup_gmail_api()


2025-10-19 23:43:25,612 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-10-19 23:43:25,618 - INFO - Gmail API authenticated successfully
2025-10-19 23:43:25,619 - INFO - Gmail API setup successful


In [11]:
def process_csv_data(csv_content):
    """Process CSV content and return DataFrame"""
    try:
        from io import StringIO
        df = pd.read_csv(StringIO(csv_content))
        
        # Clean column names (remove spaces, convert to lowercase)
        df.columns = df.columns.str.replace(' ', '_').str.lower()
        
        # Convert Business Date to datetime
        df['business_date'] = pd.to_datetime(df['business_date'])
        
        logger.info(f"Processed CSV with {len(df)} rows")
        return df
        
    except Exception as e:
        logger.error(f"Error processing CSV data: {e}")
        return None

def insert_csv_to_database(df):
    """Insert CSV data into database"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        # Prepare data for insertion
        insert_query = """
        INSERT INTO equity_data (
            business_date, managing_location, account_id, family_group_code_1, 
            legal_account_name, currency, starting_cash, daily_charges, daily_tax,
            daily_option_premiums, profit_and_loss, daily_transfers, daily_received_cash,
            daily_cash_paid, cash, open_trade_equity, total_equity, net_option_value,
            net_liquidation_value, initial_margin, maintenance_margin, excess_deficit,
            mtd_profit_and_loss, ytd_profit_and_loss, forward_cash_entries,
            forward_futures_pl, forward_charges
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        ) ON CONFLICT (business_date, account_id) DO UPDATE SET
            managing_location = EXCLUDED.managing_location,
            family_group_code_1 = EXCLUDED.family_group_code_1,
            legal_account_name = EXCLUDED.legal_account_name,
            currency = EXCLUDED.currency,
            starting_cash = EXCLUDED.starting_cash,
            daily_charges = EXCLUDED.daily_charges,
            daily_tax = EXCLUDED.daily_tax,
            daily_option_premiums = EXCLUDED.daily_option_premiums,
            profit_and_loss = EXCLUDED.profit_and_loss,
            daily_transfers = EXCLUDED.daily_transfers,
            daily_received_cash = EXCLUDED.daily_received_cash,
            daily_cash_paid = EXCLUDED.daily_cash_paid,
            cash = EXCLUDED.cash,
            open_trade_equity = EXCLUDED.open_trade_equity,
            total_equity = EXCLUDED.total_equity,
            net_option_value = EXCLUDED.net_option_value,
            net_liquidation_value = EXCLUDED.net_liquidation_value,
            initial_margin = EXCLUDED.initial_margin,
            maintenance_margin = EXCLUDED.maintenance_margin,
            excess_deficit = EXCLUDED.excess_deficit,
            mtd_profit_and_loss = EXCLUDED.mtd_profit_and_loss,
            ytd_profit_and_loss = EXCLUDED.ytd_profit_and_loss,
            forward_cash_entries = EXCLUDED.forward_cash_entries,
            forward_futures_pl = EXCLUDED.forward_futures_pl,
            forward_charges = EXCLUDED.forward_charges,
            created_at = CURRENT_TIMESTAMP;
        """
        
        # Convert DataFrame to list of tuples
        data_tuples = []
        for _, row in df.iterrows():
            data_tuple = (
                row.get('business_date'),
                row.get('managing_location'),
                row.get('account_id'),
                row.get('family_group_code_1'),
                row.get('legal_account_name'),
                row.get('currency'),
                row.get('starting_cash'),
                row.get('daily_charges'),
                row.get('daily_tax'),
                row.get('daily_option_premiums'),
                row.get('profit_and_loss'),
                row.get('daily_transfers'),
                row.get('daily_received_cash'),
                row.get('daily_cash_paid'),
                row.get('cash'),
                row.get('open_trade_equity'),
                row.get('total_equity'),
                row.get('net_option_value'),
                row.get('net_liquidation_value'),
                row.get('initial_margin'),
                row.get('maintenance_margin'),
                row.get('excess_deficit'),
                row.get('mtd_profit_and_loss'),
                row.get('ytd_profit_and_loss'),
                row.get('forward_cash_entries'),
                row.get('forward_futures_pl'),
                row.get('forward_charges')
            )
            data_tuples.append(data_tuple)
        
        # Insert data
        cursor.executemany(insert_query, data_tuples)
        conn.commit()
        
        cursor.close()
        conn.close()
        
        logger.info(f"Successfully inserted {len(data_tuples)} records to database")
        return True
        
    except Exception as e:
        logger.error(f"Error inserting CSV data: {e}")
        return False


In [12]:
def download_emails_by_date_range(start_date, end_date):
    """Download emails for a date range and process them"""
    try:
        gmail = setup_gmail_api()
        if not gmail:
            logger.error("Failed to setup Gmail API")
            return False
        
        # Convert string dates to datetime objects
        if isinstance(start_date, str):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
        if isinstance(end_date, str):
            end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        # Get emails for date range
        emails = gmail.get_emails_by_date_range(start_date, end_date)
        
        if not emails:
            logger.warning(f"No emails found for date range {start_date} to {end_date}")
            return False
        
        success_count = 0
        for date_str, message in emails:
            try:
                # Download CSV data
                csv_data = gmail.download_csv_attachment(message['id'])
                if csv_data:
                    # Process CSV
                    df = process_csv_data(csv_data.decode('utf-8'))
                    if df is not None:
                        # Insert to database
                        if insert_csv_to_database(df):
                            success_count += 1
                            logger.info(f"Successfully processed email for {date_str}")
                        else:
                            logger.error(f"Failed to insert data for {date_str}")
                    else:
                        logger.error(f"Failed to process CSV for {date_str}")
                else:
                    logger.error(f"No CSV data found for {date_str}")
            except Exception as e:
                logger.error(f"Error processing email for {date_str}: {e}")
        
        logger.info(f"Successfully processed {success_count} out of {len(emails)} emails")
        return success_count > 0
        
    except Exception as e:
        logger.error(f"Error downloading emails: {e}")
        return False

# Example usage:
download_emails_by_date_range('2025-09-26', '2025-10-10')


2025-10-19 23:43:29,321 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-10-19 23:43:29,323 - INFO - Gmail API authenticated successfully
2025-10-19 23:43:29,323 - INFO - Gmail API setup successful
2025-10-19 23:43:29,955 - INFO - Found 1 emails matching query: subject:"TIDS3121200 Equity 20250926"
2025-10-19 23:43:30,210 - INFO - Found 0 emails matching query: subject:"TIDS3121200 Equity 20250927"
2025-10-19 23:43:30,487 - INFO - Found 0 emails matching query: subject:"TIDS3121200 Equity 20250928"
2025-10-19 23:43:30,746 - INFO - Found 1 emails matching query: subject:"TIDS3121200 Equity 20250929"
2025-10-19 23:43:31,012 - INFO - Found 1 emails matching query: subject:"TIDS3121200 Equity 20250930"
2025-10-19 23:43:31,262 - INFO - Found 1 emails matching query: subject:"TIDS3121200 Equity 20251001"
2025-10-19 23:43:31,537 - INFO - Found 1 emails matching query: subject:"TIDS3121200 Equity 20251002"
2025-10-19 23:43:31,801 - INFO - Found 1 emails matching query: subjec

True

In [ ]:
def daily_automation():
    """Daily automation script to download today's email and update database"""
    try:
        logger.info("Starting daily automation process...")
        
        # Get today's date
        today = datetime.now().strftime('%Y%m%d')
        today_date = datetime.now().strftime('%Y-%m-%d')
        
        logger.info(f"Processing data for {today_date}")
        
        # Setup Gmail API
        gmail = setup_gmail_api()
        if not gmail:
            logger.error("Failed to setup Gmail API")
            return False
        
        # Download today's email
        csv_data = gmail.get_daily_equity_data(today)
        if not csv_data:
            logger.warning(f"No email found for today ({today_date})")
            return False
        
        # Process CSV data
        df = process_csv_data(csv_data)
        if df is None:
            logger.error("Failed to process CSV data")
            return False
        
        # Insert to database
        if not insert_csv_to_database(df):
            logger.error("Failed to insert data to database")
            return False
        
        # Generate updated Excel file
        excel_file = generate_excel_from_database()
        if not excel_file:
            logger.error("Failed to generate Excel file")
            return False
        
        logger.info(f"Daily automation completed successfully. Excel file: {excel_file}")
        return True
        
    except Exception as e:
        logger.error(f"Error in daily automation: {e}")
        return False

# Run daily automation
daily_automation()


In [ ]:
def get_database_stats():
    """Get statistics about the database"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        # Get total count
        cursor.execute("SELECT COUNT(*) FROM equity_data")
        total_count = cursor.fetchone()[0]
        
        # Get date range
        cursor.execute("SELECT MIN(business_date), MAX(business_date) FROM equity_data")
        date_range = cursor.fetchone()
        
        # Get latest record
        cursor.execute("SELECT business_date, account_id, total_equity FROM equity_data ORDER BY business_date DESC LIMIT 1")
        latest_record = cursor.fetchone()
        
        cursor.close()
        conn.close()
        
        stats = {
            'total_records': total_count,
            'date_range': date_range,
            'latest_record': latest_record
        }
        
        logger.info(f"Database Stats: {stats}")
        return stats
        
    except Exception as e:
        logger.error(f"Error getting database stats: {e}")
        return None

# Get database statistics
get_database_stats()


In [ ]:
get_database_stats()